In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

class AdvancedCNN(nn.Module):
    def __init__(self):
        super(AdvancedCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5, padding=2)  
        self.bn1 = nn.BatchNorm2d(10)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5, padding=2)  
        self.bn2 = nn.BatchNorm2d(20)
        self.pool = nn.AvgPool2d(2)
        self.fc1 = nn.Linear(980, 50)  # 입력 크기를 조정
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.pool(x)
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [2]:
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, ConcatDataset

# 데이터 로딩 및 전처리
def load_mnist_data(train_dirs, test_dir):
    transform = transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

    train_datasets = [datasets.ImageFolder(root=dir, transform=transform) for dir in train_dirs]
    train_dataset = ConcatDataset(train_datasets)
    test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

    return train_loader, test_loader



In [3]:
import torch
import torch.optim as optim
import torch.nn.functional as F

def train_and_evaluate(model, train_loader, test_loader, epochs=10, device='cuda'):
    optimizer = optim.Adam(model.parameters())
    criterion = torch.nn.CrossEntropyLoss()

    model.to(device)

    for epoch in range(epochs):
        model.train()
        train_loss = 0
        correct = 0
        total = 0

        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

            if batch_idx % 100 == 0:
                print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.4f}%)]\tLoss: {loss.item():.6f}')

        train_loss /= len(train_loader.dataset)
        train_accuracy = 100. * correct / total
        print(f'\nTraining set: Average loss: {train_loss:.4f}, Accuracy: {correct}/{total} ({train_accuracy:.0f}%)\n')

        # Evaluate on test data
        model.eval()
        test_loss = 0
        correct = 0
        with torch.no_grad():
            for data, target in test_loader:
                data, target = data.to(device), target.to(device)
                output = model(data)
                test_loss += criterion(output, target).item()
                _, predicted = torch.max(output.data, 1)
                correct += (predicted == target).sum().item()

        test_loss /= len(test_loader.dataset)
        test_accuracy = 100. * correct / len(test_loader.dataset)
        print(f'Test set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({test_accuracy:.4f}%)\n')

In [4]:
# 실행
train_dirs = ['C:/Users/koo/my_workspace/privacy_term_project/random_pertubation_image_by_adv', 
             'C:/Users/koo/my_workspace/privacy_term_project/mnist_btm_hash/training' ]

test_dir = 'C:/Users/koo/my_workspace/privacy_term_project/mnist_btm_hash/testing'
model = AdvancedCNN()
train_loader, test_loader = load_mnist_data(train_dirs, test_dir)
train_and_evaluate(model, train_loader, test_loader, epochs=10)
torch.save(model.state_dict(), 'advanced_mnist_cnn_by_ADT.pth') # 모델 저장

Train Epoch: 0 [0/68809 (0.0000%)]	Loss: 2.335504
Train Epoch: 0 [6400/68809 (9.2937%)]	Loss: 0.240254
Train Epoch: 0 [12800/68809 (18.5874%)]	Loss: 0.184503
Train Epoch: 0 [19200/68809 (27.8810%)]	Loss: 0.329387
Train Epoch: 0 [25600/68809 (37.1747%)]	Loss: 0.098851
Train Epoch: 0 [32000/68809 (46.4684%)]	Loss: 0.161556
Train Epoch: 0 [38400/68809 (55.7621%)]	Loss: 0.124304
Train Epoch: 0 [44800/68809 (65.0558%)]	Loss: 0.032953
Train Epoch: 0 [51200/68809 (74.3494%)]	Loss: 0.090062
Train Epoch: 0 [57600/68809 (83.6431%)]	Loss: 0.062456
Train Epoch: 0 [64000/68809 (92.9368%)]	Loss: 0.046784

Training set: Average loss: 0.0027, Accuracy: 65409/68809 (95%)

Test set: Average loss: 0.0009, Accuracy: 9815/10000 (98.1500%)

Train Epoch: 1 [0/68809 (0.0000%)]	Loss: 0.047550
Train Epoch: 1 [6400/68809 (9.2937%)]	Loss: 0.040358
Train Epoch: 1 [12800/68809 (18.5874%)]	Loss: 0.020080
Train Epoch: 1 [19200/68809 (27.8810%)]	Loss: 0.050708
Train Epoch: 1 [25600/68809 (37.1747%)]	Loss: 0.099611
Tra